In [1]:
import sys, pathlib

In [2]:
print(pathlib.Path('..\..').resolve())

D:\dieyepy\src\main\python


In [3]:
sys.path.append(str(pathlib.Path('..\..').resolve()))
# print(sys.path)

['d:\\dieyepy\\venv\\scripts\\python36.zip', 'C:\\Users\\User\\AppData\\Local\\Programs\\Python\\Python36\\DLLs', 'C:\\Users\\User\\AppData\\Local\\Programs\\Python\\Python36\\lib', 'C:\\Users\\User\\AppData\\Local\\Programs\\Python\\Python36', 'd:\\dieyepy\\venv', '', 'd:\\dieyepy\\venv\\lib\\site-packages', 'd:\\projects\\histomicstk', 'd:\\dieyepy\\venv\\lib\\site-packages\\win32', 'd:\\dieyepy\\venv\\lib\\site-packages\\win32\\lib', 'd:\\dieyepy\\venv\\lib\\site-packages\\Pythonwin', 'd:\\dieyepy\\venv\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\User\\.ipython', 'D:\\dieyepy\\src\\main\\python']


In [4]:
from ndarray_persist.ndarray_persist_utils import save_named_ndarrays
from slice.generator.response.patch_response_generator import PatchResponseGenerator
from slice.model.patch_image_config import PatchImageConfig
from slice.model.patch_image_source_config import PatchImageSourceConfig
from slice.patch_response_utils import collect_responses_to_named_ndarrays

In [ ]:
# 
level = 2
grid_length_0level = 128
grid_length = grid_length_0level * (level) ** 2
stride = grid_length / 2

In [ ]:
# There are several use-cases for slicing slides.
# 1) Do not generate patch_label_images but generate just patch_slide_images.
#    It can be useful when you want:
#    1.1) label patches manually in some graphics editor
#    1.2) label patches with your custom script
# 2) Generating patch_label_images together with generating patch_slide_images. Where
#    a) patch_label_image is a result of drawing annotation with label color stored in annotation
#    b) patch_slide_image is just a corresponding region from the same slide
# 3) Generate patch_label_images together with generating patch_slide_images. Where
#    a) patch_label_image is a region from some specifically stained slide
#    b) patch_slide_image is the same region from (possibly a bit shifted) corresponding H&E slide

In [ ]:
# Here we will consider the use-case 2.
# 

In [ ]:
# There are 2 types of config: PatchImageSourceConfig, PatchImageConfig.
# PatchImageSourceConfig defines params for generating patches from slide.
# It is named "source" because it initiates data-flow patch_pos->patch_geometry->patch_label_image.
# Different hooks can be placed in this data-flow such that irrelevant 
# and uninteresting positions/resulting label images will be filtered out.
# At this moment PatchResponseGenerator use such hooks that 
# it skips patch geometry if it doesn't fully contained by some roi annotation.
# You can add your hooks to this flow. For example you can add hook to filter out patch label images
# that are uninteresting in sense of class distribution. For example you may don't want fully white or
# fully black resuting label images but want only label image containing both classes.
# 
# Grid positions and label images can be filtered out in data-flow so 
# only after generating patch_label_image and confirming it as "interesting"(means not filtering it out)
# it make sense to generate corresponding patch_slide_image.
# It means that process of generating patch_slide_images depends on process 
# of generating patch_label_images. This dependence is represented with attribute 
# "dependents" in PatchImageSourceConfig.

In [ ]:
configs = [
#     config for source of patches
    PatchImageSourceConfig(
#         
        r"D:\temp\slides\slide1.mrxs",
        level,
        r"D:\temp\slides\slide1_annotations.json",
        metadata={"name": "label"}, 
        grid_length=grid_length, 
        stride=stride,
        dependents=[
            PatchImageConfig(
                r"D:\temp\slides\slide1.mrxs",
                level,
                metadata={"name": "image"}
            )
        ]
    ),
    PatchImageSourceConfig(r"D:\temp\slides\slide5.mrxs",
                           level,
                           r"D:\temp\slides\slide5_annotations.json",
                           metadata={"name": "label"}, grid_length=grid_length, stride=stride,
                           dependents=[
                               PatchImageConfig(r"D:\temp\slides\slide5.mrxs",
                                                level, metadata={"name": "image"}),
                           ]),
]

In [7]:
patch_responses = PatchResponseGenerator().create(configs)
format_str = r"{cfg.slide_path}/{cfg.level}/{cfg.grid_length}/{cfg.metadata[name]}/({pos[0]},{pos[1]})_{cfg.level}_{cfg.metadata[name]}"
named_ndarrays = collect_responses_to_named_ndarrays(patch_responses, format_str)
# save_named_ndarrays_to_hdf5(named_ndarrays, h5py_file_path, "w", verbosity=1)
# save_named_ndarrays(named_ndarrays, r"D:\temp", verbosity=1)
save_named_ndarrays(named_ndarrays, r"D:\temp\data.hdf5", verbosity=1)
# save_ndarray_groups_to_zip(pig, format_str, zip_file_path)
# save_ndarray_groups_to_folder(pig, format_str, folder_path, delete_working_folder=True)

ModuleNotFoundError: No module named 'ndarray_persist'

In [ ]:
# named_ndarrays = load_named_ndarrays_from_hdf5(h5py_file_path)
# named_ndarrays = list(named_ndarrays)